In [11]:
#data creator
import cv2
import numpy as np
import sqlite3

faceDetect=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cam=cv2.VideoCapture(0)

def InsertOrUpdate(Id, Name, Gender, Occupation, Age):
    con=sqlite3.connect("FaceBase.db")
    cmd="SELECT * FROM People WHERE ID="+str(Id)
    cursor=con.execute(cmd)
    isRecordExist=0
    for row in cursor:
        isRecordExist=1
    if(isRecordExist==1):
        cmd="UPDATE People SET Name=%r ,Gender=%r ,Occupation=%r ,Age=%r WHERE ID=%d"%(Name,Gender,Occupation,Age,int(Id))
        con.execute(cmd)
        con.commit()
        
    else:
        cmd="INSERT INTO People Values(%r,%r,%r,%r,%r)"%(Id, Name, Gender, Occupation, Age)
        con.execute(cmd)
        con.commit()
        con.close()

id1=input('Enter User ID')
name=input('Enter your Name')
gender=input('Enter your Gender')
occupation=input('Enter your Occupation')
Age=input('Enter your Age')
InsertOrUpdate(id1, name, gender, occupation, Age)

sampleNum=0
while(True):
    ret, img=cam.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=faceDetect.detectMultiScale(gray, 1.3,5)
    for(x,y,w,h) in faces:
        sampleNum=sampleNum+1
        cv2.imwrite("Dataset/User."+str(id1)+"."+str(sampleNum)+".jpg", gray[y:y+h,x:x+w])
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.waitKey(100)
    cv2.imshow("Frame", img)
    cv2.waitKey(1)
    if(sampleNum>100):
        break
cam.release()
cv2.destroyAllWindows()

Enter User ID23
Enter your Nameparth
Enter your Genderfemale
Enter your Occupationffsd
Enter your Age23


In [9]:
#trainer
import os
import cv2
import numpy as np
from PIL import Image

recognizer=cv2.face.LBPHFaceRecognizer_create()
path=os.path.abspath('dataset')

def getImagesWithID(path):
    imgPaths=[os.path.join(path,f) for f in os.listdir(path)]
    faces=[]
    Ids=[]
    for imgPath in imgPaths:
        faceImg=Image.open(imgPath).convert("L")
        faceNp=np.array(faceImg,'uint8')
        ID=int(os.path.split(imgPath)[-1].split('.')[1])
        faces.append(faceNp)
        Ids.append(ID)
        cv2.imshow("traning", faceNp)
        cv2.waitKey(10)
    return np.array(Ids), faces

Ids, faces=getImagesWithID(path)
recognizer.train(faces,Ids)
recognizer.save("recognizer/trainingData.yml")
cv2.destroyAllWindows()


In [10]:
#detector
import cv2
import numpy as np
import sqlite3
faceDetect=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cam=cv2.VideoCapture(0)
rec=cv2.face.LBPHFaceRecognizer_create()
rec.read("recognizer/trainingData.yml")

def getProfile(id):
    con=sqlite3.connect("FaceBase.db")
    cmd="SELECT * FROM People WHERE ID="+str(id)
    cursor=con.execute(cmd)
    profile=None
    for row in cursor:
        profile=row
    con.close()
    return profile

id=0
#font = cv2.cv2.InitFont(cv2.cv2.CV_FONT_HERSHEY_SIMPLEX, 1, 1, 0, 1, 1)
fontface = cv2.FONT_HERSHEY_SIMPLEX
fontcolor = (0, 255, 0)
while(True):
    ret, img=cam.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=faceDetect.detectMultiScale(gray, 1.3,5)
    for(x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
        id, conf=rec.predict(gray[y:y+h, x:x+w])
        profile=getProfile(id)
        if conf>60:
            profile=None
        if (profile!=None):
            #cv2.cv.putText(cv2.cv.fromarray(img),str(id),(x,y+h),font,255)
            cv2.putText(img, "Name:"+str(profile[1]), (x,y+h+30), fontface, 0.6, fontcolor, 2)
            cv2.putText(img, "Age:"+str(profile[4]), (x,y+h+60), fontface, 0.6, fontcolor, 2)
            cv2.putText(img, "Gender:"+str(profile[2]), (x,y+h+90), fontface, 0.6, fontcolor, 2)
            cv2.putText(img, "Occupation:"+str(profile[3]), (x,y+h+120), fontface, 0.6, fontcolor, 2)

    cv2.imshow("Frame", img)
    if(cv2.waitKey(1)==ord('q')):
        break;
cam.release()
cv2.destroyAllWindows()




20
